In [12]:
from collections import Counter
from itertools import product
import math

class DDEFeatureExtraction:
    def __init__(self):
        self.amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
        self.codons = {
            'A': 4, 'R': 6, 'N': 2, 'D': 2, 'C': 2,
            'Q': 2, 'E': 2, 'G': 4, 'H': 2, 'I': 3,
            'L': 6, 'K': 2, 'M': 1, 'F': 2, 'P': 4,
            'S': 6, 'T': 4, 'W': 1, 'Y': 2, 'V': 4
        }
        self.total_codons = sum(self.codons.values())

    def calculate_dde_features(self, sequence):
        """
        Calculate the DDE features for a given protein sequence.
    
        :param sequence: The protein sequence as a string.
        :return: A list of DDE feature values.
        """
        dpc_features = self.calculate_dpc_features(sequence)
        dde_features = []
    
        for x, y in product(self.amino_acids, repeat=2):
            dipeptide = x + y
            dpc = dpc_features.get(dipeptide, 0)  # Use get() to handle missing dipeptides
            tm = self.calculate_tm(x, y)
            tv = self.calculate_tv(tm, len(sequence))
            dde = self.calculate_dde(dpc, tm, tv)
            dde_features.append(dde)
    
        return dde_features

    def calculate_dpc_features(self, sequence):
        """
        Calculate the dipeptide composition (DPC) features.

        :param sequence: The protein sequence as a string.
        :return: A dictionary of DPC feature values.
        """
        possible_dipeptides = [''.join(dp) for dp in product(self.amino_acids, repeat=2)]
        dipeptide_counts = Counter(sequence[i:i + 2] for i in range(len(sequence) - 1))
        total_dipeptides = sum(dipeptide_counts.values())
        dpc = {dp: dipeptide_counts[dp] / total_dipeptides for dp in possible_dipeptides}
        return dpc

    def calculate_tm(self, x, y):
        """
        Calculate the theoretical mean (Tm) for amino acids x and y.

        :param x: The first amino acid.
        :param y: The second amino acid.
        :return: The theoretical mean value.
        """
        cx = self.codons[x]
        cy = self.codons[y]
        tm = (cx / self.total_codons) * (cy / self.total_codons)
        return tm

    def calculate_tv(self, tm, sequence_length):
        """
        Calculate the theoretical variance (Tv) based on Tm and sequence length.

        :param tm: The theoretical mean value.
        :param sequence_length: The length of the protein sequence.
        :return: The theoretical variance value.
        """
        tv = (tm * (1 - tm)) / (sequence_length - 1)
        return tv

    def calculate_dde(self, dpc, tm, tv):
        """
        Calculate the deviation from expected dipeptide composition (DDE).

        :param dpc: The dipeptide composition value.
        :param tm: The theoretical mean value.
        :param tv: The theoretical variance value.
        :return: The DDE value.
        """
        if tv == 0:
            return 0
        else:
            dde = (dpc - tm) / math.sqrt(tv)
            return dde

In [13]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, matthews_corrcoef
from sklearn.model_selection import LeaveOneOut
import numpy as np

# Load the dataset
data = pd.read_excel('../data/Final_2Sm_modified_with_sequences.xlsx')

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit the encoder to the folding_type column and transform it to numeric labels
data['folding_type'] = label_encoder.fit_transform(data['folding_type'])

# Now, when you extract labels for model training:
labels = data['folding_type'].values

# Initialize the FeatureExtraction class
feature_extraction = DDEFeatureExtraction()

# Feature extraction using AAC with length
features = np.array([feature_extraction.calculate_dde_features(seq) for seq in data['sequence']])

# SVM with Leave-One-Out Cross-Validation (LOOCV)
loo = LeaveOneOut()
y_true, y_pred = [], []
for train_index, test_index in loo.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)
    y_pred.append(clf.predict(X_test)[0])
    y_true.append(y_test[0])

In [10]:
# Calculate and display the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)
print("Confusion Matrix DDE:")
print(conf_matrix)

# Calculate and display the accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f"\nAccuracy (ACC): {accuracy:.2f}")

# Calculate and display the Matthews Correlation Coefficient (MCC)
mcc = matthews_corrcoef(y_true, y_pred)
print(f"Matthews Correlation Coefficient (MCC): {mcc:.2f}")

# Generate classification report
report = classification_report(y_true, y_pred, zero_division=0)
print("\nClassification Report:")
print(report)

Confusion Matrix DDE:
[[66 23]
 [25 27]]

Accuracy (ACC): 0.66
Matthews Correlation Coefficient (MCC): 0.26

Classification Report:
              precision    recall  f1-score   support

           0       0.73      0.74      0.73        89
           1       0.54      0.52      0.53        52

    accuracy                           0.66       141
   macro avg       0.63      0.63      0.63       141
weighted avg       0.66      0.66      0.66       141


# Random Forest Classifier implementation with hyperparameter tuning

In [16]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Define the parameter grid
rf_param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Initialize the Random Forest classifier
rf = RandomForestClassifier(random_state=42)

# Grid search with cross validation setup
grid_search = GridSearchCV(estimator=rf, param_grid=rf_param_grid, cv=3, scoring='accuracy')

# Fit the grid search to find the best parameters
grid_search.fit(features, labels)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best parameters: ", best_params)
print("Best score: ", best_score)

# Using the best parameters with LOOCV
best_rf = RandomForestClassifier(**best_params, random_state=42)
loo = LeaveOneOut()
y_true, y_pred = [], []

for train_index, test_index in loo.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = labels[train_index], labels[test_index]
    best_rf.fit(X_train, y_train)
    y_pred.append(best_rf.predict(X_test)[0])
    y_true.append(y_test[0])

Best parameters:  {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Best score:  0.8085106382978723


In [17]:
# Evaluate the model
# Calculate and display the confusion matrix
from ClassificationMatrix import ClassificationMatrix

cm = ClassificationMatrix(y_true, y_pred, 'DDE')
cm.evaluate()

Confusion Matrix: $GDPC
[[83  6]
 [21 31]]

Accuracy (ACC): 0.81
Matthews Correlation Coefficient (MCC): 0.58

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.93      0.86        89
           1       0.84      0.60      0.70        52

    accuracy                           0.81       141
   macro avg       0.82      0.76      0.78       141
weighted avg       0.81      0.81      0.80       141
